In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [3]:
!ls ../input/10-monkey-species/

In [4]:
train_dir = "../input/10-monkey-species/training/training"
valid_dir = "../input/10-monkey-species/validation/validation"
label_file = "../input/10-monkey-species/monkey_labels.txt"
print(os.path.exists(train_dir))
print(os.path.exists(valid_dir))
print(os.path.exists(label_file))

print(os.listdir(train_dir))
print(os.listdir(valid_dir))

In [5]:
labels = pd.read_csv(label_file, header=0)
print(labels)

In [6]:
# 因为resnet处理的224，因此要改
height = 224
width = 224
channels = 3
# 图形变大了，我们改小batch_size
batch_size = 24
num_classes = 10

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    # 这里也要改为resnet50，resnet50是karas中，为tf，torch都做了适配
    preprocessing_function = keras.applications.resnet50.preprocess_input,
    rotation_range = 40,  # 随即旋转一个角度
    width_shift_range = 0.2,  # 偏移
    height_shift_range = 0.2,  # 垂直方向偏移
    shear_range = 0.2,  # 剪切强度
    zoom_range = 0.2,  # 缩放强度
    horizontal_flip = True,  # 水平随即旋转
    fill_mode = 'nearest',  # 填充
)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size = (height, width),  # 生成图片多大
                                                   batch_size = batch_size,  # 生成的图片以多少张为一组
                                                   seed = 7,
                                                   shuffle = True,
                                                   class_mode = "categorical"  # one-hot编码的一种模式
)
# 这里也需要修改
valid_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size = (height, width),
                                                    batch_size = batch_size,
                                                    seed = 7,
                                                    shuffle = False,
                                                    class_mode = "categorical")
train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num, valid_num)

In [8]:
for i in range(2):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)
    print('-'*50)

In [24]:
resnet50_fine_tune = keras.models.Sequential()
# resnet有1000个分类，我们只有10类，最后一层要去掉
# 最后的输出是三维矩阵，而不是一维的，通过pooling = 'avg'解决这个问题
# pooling size 是（2，2）的时候是大小减半，而pooling size恰好等于图像大小的时候，就可以降维
resnet50_fine_tune.add(keras.applications.ResNet50(include_top = False,
                                                   pooling = 'avg',
                                                   weights = 'imagenet'  # weights = 'imagenet'就会下载imagenet，在这个初始化好的基础上去训练
))
# 加一个全连接层num_classes值为10
resnet50_fine_tune.add(keras.layers.Dense(num_classes, activation = 'softmax'))
print(resnet50_fine_tune.layers)
print(len(resnet50_fine_tune.layers))
resnet50_fine_tune.layers[0].trainable = False  # 这一层不进行训练

resnet50_fine_tune.compile(loss="categorical_crossentropy",
                           optimizer="sgd", 
                           metrics=['accuracy'])
resnet50_fine_tune.summary()

In [10]:
epochs = 10
history = resnet50_fine_tune.fit(train_generator,
                                steps_per_epoch = train_num // batch_size,
                                epochs = epochs,
                                validation_data = valid_generator,
                                validation_steps = valid_num // batch_size)

In [11]:
def plot_learning_curves(history, label, epcohs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 0, 2)

In [12]:
# 另外一种实现
resnet50 = keras.applications.ResNet50(include_top = False,
                                       pooling = 'avg',
                                       weights = 'imagenet')
resnet50.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(resnet50)

In [20]:
print(len(resnet50.layers))
# 设置只有最后的5层可以训练,前面的172层不训练
for layer in resnet50.layers[0:-5]:  # 0到最后5层
    layer.trainable = False

resnet50_new = keras.models.Sequential([
    resnet50,
    keras.layers.Dense(num_classes, activation = 'softmax'),
])
resnet50_new.compile(loss="categorical_crossentropy",
                     optimizer="sgd",
                     metrics=['accuracy'])
resnet50_new.summary()
print(len(resnet50.layers))
# 可以看到params大大增加了

In [21]:
# 增加了以后准确率变化了，两个原因：1.可训练参数变多，2。增加的层进行了调整
epochs = 10
history = resnet50_new.fit_generator(train_generator,
                                     steps_per_epoch = train_num // batch_size,
                                     epochs = epochs,
                                     validation_data = valid_generator,
                                     validation_steps = valid_num // batch_size)

In [22]:
plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 0, 2)